In [1]:
from pathlib import Path
from ragdaemon.daemon import Daemon

annotators = {
    "hierarchy": {},
    "chunker_llm": {},
    # "call_graph": {},
    # "summarizer": {},
    "diff": {},
}

sample_dir = Path.cwd()
daemon = Daemon(
    cwd=sample_dir, 
    annotators=annotators, 
    verbose=2
)
await daemon.update()

Initialized empty graph.
Initializing annotators: ['hierarchy', 'chunker_llm', 'diff']...


Chunking files...: 100%|██████████| 53/53 [00:12<00:00,  4.26it/s]

Saved updated graph to /Users/granthawkins/.mentat/ragdaemon/ragdaemon-ragdaemon.json


In [5]:
res = daemon.search("checksum")
for item in res[:20]:
    print(item["id"], item["distance"])

[{'checksum': '251c05e23413db161aa29437f0a2dfc3', 'distance': 0.0}, {'checksum': '04f97af30345c81aa8738a11f0c62129', 'distance': 0.1094384372513798}, {'checksum': 'cf667896778beea35bd924b2d87c9d15', 'distance': 0.20102943302176224}, {'checksum': '3bd4ddb264db0a66d791974c26cc86fc', 'distance': 0.22612771931484454}, {'checksum': 'e2d50d4d16590b0a1d5d28ce971d359f', 'distance': 0.22613115143282891}, {'checksum': '0616f7aa2d675d4cb743cb32e1982848', 'distance': 0.2581375806755489}, {'checksum': '0223e5bfc9508c92f4c595c727dd6071', 'distance': 0.2950812919579382}, {'checksum': 'c54b960b439036743b52d958c6d54916', 'distance': 0.31875416971760184}, {'checksum': 'c4d480bc0ea13a556ab30b4fcd726405', 'distance': 0.4379991436485372}, {'checksum': 'a806f165b4ad4f01b22a0cb2cf58b967', 'distance': 0.520479910750369}, {'checksum': '09e0c7d746907944f7d089952a2cc646', 'distance': 0.5302764506407515}, {'checksum': '014f1805de5ef0b088689879b446ba85', 'distance': 0.7089103670794901}, {'checksum': '3ee0ddc90fd05

In [6]:
import json

backup_dir = Path.home() / "Desktop" / "chroma-backup"
n_ids = len(all_ids["ids"])
batch_size = 10_000
n_batches = n_ids // batch_size + 1
for i in range(n_batches):
    print(f"Processing batch {i+1}/{n_batches}")
    _start = i * batch_size
    _end = min((i + 1) * batch_size, n_ids)
    batch = all_ids["ids"][_start:_end]
    data = daemon.db.get(ids=batch, include=["embeddings", "documents", "metadatas"])
    with open(backup_dir / f"batch_{i}.json", "w") as f:
        json.dump(data, f)


Processing batch 1/15


KeyboardInterrupt: 

In [2]:
import os
import dotenv

from ragdaemon.utils import basic_auth

import chromadb
from chromadb.config import Settings

dotenv.load_dotenv()

host = os.environ["CHROMA_SERVER_HOST"]
port = int(os.environ.get("CHROMA_SERVER_HTTP_PORT", 443))
username = os.environ["CHROMA_SERVER_USERNAME"]
password = os.environ["CHROMA_SERVER_PASSWORD"]
_client = chromadb.HttpClient(
    host=host,
    port=port,
    ssl=port == 443,
    headers={"Authorization": basic_auth(username, password)},
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
)

_client.list_collections()

[Collection(name=ragdaemon-0.6-text-embedding-3-large-embeddings-dictionary),
 Collection(name=ragdaemon-0.4-text-embedding-3-large),
 Collection(name=ragdaemon-0.5-text-embedding-3-large)]

In [3]:
for collection in [
    # "ragdaemon-0.6-text-embedding-3-large-ragdaemon",
    # "ragdaemon-0.5-text-embedding-3-large-ragdaemon",
    "ragdaemon-0.6-text-embedding-3-large-embeddings-dictionary",
]:
    _client.delete_collection(collection)

Exception: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.24.0 (Ubuntu)</center>
</body>
</html>


In [5]:
_client.list_collections()
# _client.get_or_create_collection("test-collection")
# _client.delete_collection("test-collection")

[Collection(name=ragdaemon-0.4-text-embedding-3-large),
 Collection(name=ragdaemon-0.5-text-embedding-3-large),
 Collection(name=ragdaemon-0.6-text-embedding-3-large-embeddings-dictionary)]